In [2]:
import serial # damit ich von Serial-Connection Daten lesen kann 
import time # weil wir Zeit zum Warten brauchen
import pandas as pd # zum Verarbeiten von time series
import matplotlib.pyplot as plt # für Heatmaps
import seaborn as sns # für Heatmaps

In [5]:
dataDemo = []  # List to store incoming data rows

# COM Port abgreifen mit der Baudrate
with serial.Serial('COM6', baudrate=57600) as ser:
    time.sleep(5)  # Anfangsdelay (hat nur mit 5s )
    print("starting ...") # damit wir überprüfen können, dass anfängt

    readValue = "o" # hier Unterprogramm vom Arduino Code übergeben


    # ser.write schreibt zum Arduino über serial connection 
    # der Befehl macht aus dem Unterprogramm des Commands vom Python-String 
    # einen Byte-String, die der Arduino dann verwerten kann, also der 
    # hier eingegebene String namens readValue wird codiert nach dem utf-8 Protokoll


    ser.write(readValue.encode("utf-8"))    

    while True:  # ewige Schleife

        # ser.readline() liest aus serieller Schnittstelle so viele Bytes bis 
        # \n kommt (also bis man im Arduino Code eine neue Zeile gestartet hat)
        # (deshalb überall println im Arduino)

        line = ser.readline()          

        # Wenn in der Zeile etwas drinnen steht (Länge > 0) 
        # dann wird response generiert -> Info vom Arduino, kommt jetzt als 
        # Byte-String daher, der muss wieder vom utf-8 Protokoll DE-codiert werden
        # .strip(), um \r\n nicht mitauszugeben -> entfernt sogenannte 
        # "white space characters"

        if len(line) > 0:                                   
            response = line.decode("utf-8").strip()         
            print(response)
            
            # wenn im Arduino Code bei "exit" angelangt ist, unterbricht Programm
            # (wenn man das nicht einbaut, läuft Python-Ausleseprogramm immer weiter)

            if response == "exit":
                break

            # ebenso unterbrechen, wenn an erster Stelle ein # steht 
            # (weil alle str in unserem Arduino Code so beginnen)

            if response[0] == "#":
                continue

            try:
                # response in einen float umwandeln und , als Seperator  
                # dann oben erstellter Liste hinzufügen (lists of lists entsteht)
                row = [float(element) for element in response.split(",")]
                dataDemo.append(row)
            except ValueError:
                # wenn Teil unter try nicht geht, wird aktuelle Zeile und 
                # Error Message ausgegeben
                print("Error parsing line:", response)

if dataDemo:
    df_Demo = pd.DataFrame(dataDemo)  # Liste von Listen in einen Dataframe konvertieren
    print(df_Demo)
    df_Demo.to_csv('Demo Dataframe.csv', index=False, header=False) # und als CSV speichern
else:
    print("No data collected")  # wenn nicht geht, Error message ausgeben


SerialException: could not open port 'COM6': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [13]:
df = pd.DataFrame(dataDemo, columns = 
                  ['hDeg', 'vDeg', 'AbmientT', 'ObjectT', 'Clear', 'Red', 'Green', 'Blue', 'IR'])
df

,hDeg,vDeg,AbmientT,ObjectT,Clear,Red,Green,Blue,IR
0,0.0,45.0,29.63,26.63,54.0,46.0,28.0,24.0,384.0
1,0.0,50.0,29.77,26.63,50.0,43.0,26.0,22.0,347.0
2,0.0,55.0,30.03,26.67,45.0,38.0,24.0,20.0,315.0
3,0.0,60.0,30.17,26.65,40.0,34.0,21.0,18.0,277.0
4,0.0,65.0,30.39,26.64,38.0,31.0,20.0,17.0,255.0
...,...,...,...,...,...,...,...,...,...
337,180.0,110.0,24.83,28.03,26.0,18.0,17.0,14.0,114.0
338,180.0,115.0,24.77,28.03,25.0,17.0,16.0,13.0,105.0
339,180.0,120.0,24.71,28.05,25.0,16.0,15.0,13.0,94.0
340,180.0,125.0,24.65,28.03,24.0,15.0,15.0,12.0,88.0
